## Project Structure

In [2]:
import sys
sys.version

'3.9.1 (default, May 28 2021, 21:38:31) \n[GCC 9.3.0]'

In [23]:
!pipenv install xgboost

Installing xgboost...
Adding xgboost to Pipfile's [packages]...
✔ Installation Succeeded 
Pipfile.lock (16c735) out of date, updating to (860296)...
Locking [dev-packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! 
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! 
Updated Pipfile.lock (860296)!
Installing dependencies from Pipfile.lock (860296)...
  🐍   ▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉ 0/0 — 00:00:00


In [22]:
import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn import metrics

ModuleNotFoundError: No module named 'xgboost'

## Zindi: PAYGo

In [4]:
train = pd.read_csv("../../data/Train.csv", sep=",")
metadata = pd.read_csv("../../data/metadata.csv", sep=",")
test = pd.read_csv("../../data/Test.csv", sep=",")

/home/aumaron/.local/share/virtualenvs/zindi_payg-FXkRANRI/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
metadata[metadata["ID"].isin(list(train["ID"].values))]

,ID,RegistrationDate,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,Term,TotalContractValue,ExpectedTermDate,FirstPaymentDate,LastPaymentDate
0,ID_K00S4N4,2015-12-10 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Male,41.0,Mount Kenya Region,Embu,Other,d_light,364,14740.0,2016-12-08 00:00:00,2015-12-10 09:52:35,2016-10-23 04:52:30
2,ID_102CV85,2015-12-18 00:00:00,2000,2018-03-29 10:14:58,35.0,FINANCED,DAILY,35,7,Female,48.0,Nairobi Region,Makueni,Business,d_light,392,29480.0,2017-01-13 00:00:00,2015-12-18 06:22:34,2017-02-01 15:23:44
3,ID_HXBJFHB,2015-11-25 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Female,43.0,NaN,UNKNOWN,Teacher,d_light,364,14740.0,2016-11-23 00:00:00,2015-11-25 13:25:57,2017-05-22 16:46:54
4,ID_3K9VZ5J,2015-12-02 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Female,56.0,Mount Kenya Region,Kirinyaga,Other,d_light,364,14740.0,2016-11-30 00:00:00,2015-12-05 10:34:32,2017-05-12 16:50:52
6,ID_6YU5YRB,2015-12-10 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Male,56.0,Mount Kenya Region,Meru,Other,d_light,364,14740.0,2016-12-08 00:00:00,2015-12-10 15:15:16,2016-12-11 11:48:01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37338,ID_OKJ96OW,2020-02-20 18:26:52,2400,NaN,0.0,FINANCED,DAILY,50,3,Female,38.0,Western,Bungoma,Farmer,d_light,240,14400.0,2020-10-17 18:26:52,2020-02-20 18:26:58,2020-12-19 07:16:44
37339,ID_5J9OMUP,2020-02-20 18:30:59,2400,NaN,0.0,FINANCED,DAILY,50,3,Male,23.0,Nyanza,Siaya,Labourer,d_light,240,14400.0,2020-10-17 18:30:59,2020-02-20 18:31:06,2021-05-18 16:07:49
37340,ID_J4I8Z2V,2020-02-20 18:42:26,3000,NaN,0.0,FINANCED,DAILY,40,3,Male,31.0,Coast Region,Kwale,Teacher,d_light,340,16600.0,2021-01-25 18:42:26,2020-02-20 18:42:33,2021-04-21 15:59:37
37341,ID_GHHAQ9D,2020-02-20 18:46:30,2400,NaN,0.0,FINANCED,DAILY,50,3,Male,27.0,NaN,UNKNOWN,Labourer,d_light,240,14400.0,2020-10-17 18:46:30,2020-02-20 18:46:37,2021-04-24 19:06:01


In [6]:
train.columns
# metadata

Index(['ID', 'TransactionDates', 'PaymentsHistory', 'm1', 'm2', 'm3', 'm4',
       'm5', 'm6'],
      dtype='object')

In [7]:
train['TransactionDates'].values

array(["['04-2018', '05-2018', '06-2018', '07-2018', '08-2018', '09-2018', '10-2018', '11-2018', '12-2018', '02-2019', '03-2019', '04-2019', '05-2019', '06-2019', '07-2019', '08-2019', '09-2019', '10-2019', '11-2019', '12-2019', '01-2020', '02-2020', '03-2020', '04-2020', '05-2020', '06-2020', '07-2020', '08-2020', '09-2020', '10-2020', '11-2020']",
       "['04-2018', '05-2018', '06-2018', '07-2018', '08-2018', '11-2018', '12-2018', '01-2019', '02-2019', '03-2019', '04-2019', '05-2019', '06-2019', '07-2019', '08-2019', '09-2019', '10-2019', '11-2019', '12-2019', '01-2020', '02-2020', '03-2020', '04-2020', '05-2020', '06-2020', '07-2020', '08-2020', '09-2020', '10-2020', '11-2020']",
       "['02-2020', '03-2020', '04-2020', '05-2020', '06-2020', '07-2020']",
       ...,
       "['01-2020', '02-2020', '03-2020', '04-2020', '05-2020', '06-2020', '07-2020', '08-2020', '09-2020']",
       "['02-2018', '03-2018', '04-2018', '05-2018', '06-2018', '07-2018', '08-2018', '09-2018']",
       "[

In [8]:
print(train["ID"].nunique())         # Each contract is unique
print(metadata["ID"].nunique())      # Each contract is unique

28007
37343


In [9]:
metadata.shape

(37343, 20)

In [10]:
train_merged = metadata.merge(train[['ID', 'TransactionDates', 'PaymentsHistory',
                      'm1', 'm2', 'm3', 'm4', 'm5', 'm6']], how="left", on="ID") 

In [11]:
train_merged[train_merged["ID"] == "ID_K00S4N4"]["TransactionDates"].values

array(["['12-2015', '01-2016', '02-2016', '03-2016', '04-2016']"],
      dtype=object)

In [12]:
train_merged[train_merged["ID"] == "ID_K00S4N4"]["PaymentsHistory"].values

array(['[3050.0, 1050.0, 910.0, 1050.0, 1050.0]'], dtype=object)

In [13]:
train_merged[train_merged["ID"] == "ID_K00S4N4"][['m1', 'm2', 'm3', 'm4', 'm5', 'm6']].values

array([[1225., 1050., 1190.,  525., 1750., 1890.]])

In [14]:
train_merged.iloc[:, :-8]

,ID,RegistrationDate,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,Term,TotalContractValue,ExpectedTermDate,FirstPaymentDate,LastPaymentDate
0,ID_K00S4N4,2015-12-10 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Male,41.0,Mount Kenya Region,Embu,Other,d_light,364,14740.0,2016-12-08 00:00:00,2015-12-10 09:52:35,2016-10-23 04:52:30
1,ID_6L67PAA,2015-12-09 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Male,33.0,Coast Region,Kilifi,Other,d_light,364,14740.0,2016-12-07 00:00:00,2015-12-09 13:14:03,2020-05-24 15:32:18
2,ID_102CV85,2015-12-18 00:00:00,2000,2018-03-29 10:14:58,35.0,FINANCED,DAILY,35,7,Female,48.0,Nairobi Region,Makueni,Business,d_light,392,29480.0,2017-01-13 00:00:00,2015-12-18 06:22:34,2017-02-01 15:23:44
3,ID_HXBJFHB,2015-11-25 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Female,43.0,NaN,UNKNOWN,Teacher,d_light,364,14740.0,2016-11-23 00:00:00,2015-11-25 13:25:57,2017-05-22 16:46:54
4,ID_3K9VZ5J,2015-12-02 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Female,56.0,Mount Kenya Region,Kirinyaga,Other,d_light,364,14740.0,2016-11-30 00:00:00,2015-12-05 10:34:32,2017-05-12 16:50:52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37338,ID_OKJ96OW,2020-02-20 18:26:52,2400,NaN,0.0,FINANCED,DAILY,50,3,Female,38.0,Western,Bungoma,Farmer,d_light,240,14400.0,2020-10-17 18:26:52,2020-02-20 18:26:58,2020-12-19 07:16:44
37339,ID_5J9OMUP,2020-02-20 18:30:59,2400,NaN,0.0,FINANCED,DAILY,50,3,Male,23.0,Nyanza,Siaya,Labourer,d_light,240,14400.0,2020-10-17 18:30:59,2020-02-20 18:31:06,2021-05-18 16:07:49
37340,ID_J4I8Z2V,2020-02-20 18:42:26,3000,NaN,0.0,FINANCED,DAILY,40,3,Male,31.0,Coast Region,Kwale,Teacher,d_light,340,16600.0,2021-01-25 18:42:26,2020-02-20 18:42:33,2021-04-21 15:59:37
37341,ID_GHHAQ9D,2020-02-20 18:46:30,2400,NaN,0.0,FINANCED,DAILY,50,3,Male,27.0,NaN,UNKNOWN,Labourer,d_light,240,14400.0,2020-10-17 18:46:30,2020-02-20 18:46:37,2021-04-24 19:06:01


In [15]:
train_merged["SupplierName"].unique()

array(['d_light'], dtype=object)

In [18]:
# train_merged['days_elapsed_after_term_completion'] = 

In [19]:
train_merged["ID"].nunique()

37343

In [20]:
# Creating a baseline model to get the base RMSE score
# Creating an XGBoost model and predicting M1 only at the moment for completing the pipeline

### Regressor model

In [ ]:
def predict_model(train_array, target):
    model = xgb.XGBRegressor(use_label_encoder=False,  # Will be deprecated
                             verbosity=0,
                             objective="reg:squarederror",
                             reg_lambda=0,
                             reg_alpha=50
                            )
    from sklearn.model_selection import train_test_split
    for i in range(1, 11):
        X_train, X_temst, y_train, y_test = train_test_split(train_array, target, test_size=0.25)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        from sklearn.metrics import mean_absolute_percentage_error
        from sklearn.metrics import mean_squared_error
        feature_importance = model.feature_importances_
        # Mape
        mape = mean_absolute_percentage_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)

    return y_pred, mape, mse, feature_importance